In [1]:
import numpy as np
import pandas as pd

from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr
    
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [3]:
ds = load_dataset("beanham/spatial_union_dataset")
test=ds['test']
gt=np.array(test['label'])

In [5]:
results=[]
metric_values = ['worst_single', 'best_single', 'worst_comb', 'best_comb']
for model in ['4o_mini', '4o','qwen_plus']:
    for value in metric_values:
        few_shot=np.load(f'base/{model}_correction/{model}_{value}_few_shot_with_heur_value_angle_area_correction.npy')
        few_shot=post_processing(few_shot)
        few_metrics=metric_calculation(few_shot, gt)        
        results.append([model, value, 'few_shot', round(few_metrics[0],3), few_metrics[1]])
results=pd.DataFrame(results, columns=['model', 'value', 'prompt', 'acc', 'f1'])        

In [6]:
results

,model,value,prompt,acc,f1
0,4o_mini,worst_single,few_shot,0.877,0.855033
1,4o_mini,best_single,few_shot,0.945,0.939867
2,4o_mini,worst_comb,few_shot,0.925,0.919218
3,4o_mini,best_comb,few_shot,0.957,0.953460
4,4o,worst_single,few_shot,0.897,0.879799
5,4o,best_single,few_shot,0.935,0.927746
6,4o,worst_comb,few_shot,0.947,0.941940
7,4o,best_comb,few_shot,0.942,0.936193
8,qwen_plus,worst_single,few_shot,0.920,0.910758
9,qwen_plus,best_single,few_shot,0.955,0.951799
